In [1]:
def _raster_paths(df_features, ftype, tile = None, year = None):
    mask = (df_features['type'] == ftype)
    path_col = 'path'
    if ftype == 'landsat':
        df_features['ypath'] = df_features[mask]['path'].apply(lambda p: p.replace('{tile}', tile).replace('{year}', str(year)))
        path_col = 'ypath'
    ids_list = list(df_features[mask]['idx'])
    raster_files = list(df_features[mask][path_col])
    return raster_files, ids_list

In [2]:
def _get_static_layers_info(tiles_gpkg, features_csv, tile):
    tiles = gpd.read_file(tiles_gpkg)
    df_features = pd.read_csv(features_csv)
    min_x, _, _, max_y = tiles[tiles['TILE'] == tile].iloc[0].geometry.bounds
    static_files, _ = _raster_paths(df_features, 'static')
    gidal_ds = gdal.Open(static_files[0]) # It is assumed to be the same for all static layers
    gt = gidal_ds.GetGeoTransform()
    gti = gdal.InvGeoTransform(gt)
    x_off_s, y_off_s = gdal.ApplyGeoTransform(gti, min_x, max_y)
    x_off_s, y_off_s = int(x_off_s), int(y_off_s)
    return static_files, x_off_s, y_off_s

In [3]:
import numpy as np
import skmap_bindings
from skmap.misc import find_files
import time
import tempfile
from pathlib import Path
from osgeo import gdal, gdal_array
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

x_off = 2
y_off = 2
x_size = 4000
y_size = 4000
n_pix = x_size * y_size

tile = '120E_46N'
file_loc = f'http://192.168.49.35:8333/prod-landsat-ard2/{tile}/seasconv/blue_glad.SeasConv.ard2_m_30m_s_20000101_20000228_go_epsg.4326_v20230908.tif'
conf_GDAL = {"CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_VERSION": "1.0"}
lon_lat = np.zeros((2,n_pix), dtype=np.float32)
n_threads = 96

skmap_bindings.getLatLonArray(lon_lat, n_threads, conf_GDAL, file_loc,
               x_off, y_off, x_size, y_size)
latitude = lon_lat[1,:].copy()

In [4]:
tiles_gpkg = 'ard2_final_status.gpkg'
features_csv = './model/features.csv'
static_files, x_off_s, y_off_s = _get_static_layers_info(tiles_gpkg, features_csv, tile)
n_static = len(static_files)
idx_static = range(n_static)
dtm_file = str(find_files('./static', '*.vrt')[47])
print(dtm_file)
bands_list = [1,]
elevation = np.zeros((1,n_pix), dtype=np.float32)
skmap_bindings.readData(elevation, n_threads, [dtm_file,], [0,], x_off_s, y_off_s, x_size, y_size, bands_list, conf_GDAL)
print(elevation)

static/dtm.bareearth_ensemble_p10_30m_s_2018_go_epsg4326_v20230210.vrt
[[12741. 12752. 12762. ...  7643.  7633.  7630.]]


In [5]:
days_of_year = [20., 140., 200.]
result_indices = [0, 1, 2]
geom_temp = np.zeros((len(days_of_year), n_pix), dtype=np.float32)
result_scaling = 100.
a = 37.03043
b = -15.43029
elev_scaling = 0.1
skmap_bindings.computeGeometricTemperature(geom_temp, n_threads, latitude, elevation, elev_scaling, a, b, result_scaling, result_indices, days_of_year)
print(geom_temp)

[[-495. -496. -497. ... -106. -105. -105.]
 [1202. 1202. 1201. ... 1564. 1565. 1565.]
 [1761. 1760. 1760. ... 2114. 2114. 2114.]]
